# Loading and Cleaning the Data

## Step 1: Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt 
import pickle

## Step 2: Reading in the Turnstile Data

In [32]:
# Define the weeks to retrieve from the MTA database
def list_weeks(list_of_dates):
    weeks = []
    for date in list_of_dates:
        week_string = date.strftime('%y%m%d')
        weeks.append(week_string)
    return weeks        

# We want to pull in data from a week each month January - June in 2018 and 2019
# dates_
# dates = list(map(dt.datetime,))
dates = [dt.datetime(2018,1,13),dt.datetime(2018,2,10),dt.datetime(2018,3,10),dt.datetime(2018,4,14),\
         dt.datetime(2018,5,12),dt.datetime(2018,6,9),\
         dt.datetime(2019,1,12),dt.datetime(2019,2,9),dt.datetime(2019,3,9),dt.datetime(2019,4,13),\
        dt.datetime(2019,5,11),dt.datetime(2019,6,8)]
weeks = list_weeks(dates)
weeks

['180113',
 '180210',
 '180310',
 '180414',
 '180512',
 '180609',
 '190112',
 '190209',
 '190309',
 '190413',
 '190511',
 '190608']

In [33]:
# Read in the data for the desired weeks
def readTurnstileData(week_string):
    """
    This function reads in data from an online MTA Turnstile dataset into a DataFrame
    ---
    input: link to dataset
    output: DataFrame
    """
    cols = ['control_area','unit','scp','station','line_name','division','date','time',
        'desc','entries','exits']
    
    link = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_' + week_string + '.txt'
    turnstile_data = pd.read_csv(link, header = 0, names = cols)
    return turnstile_data

# Create a single DataFrame containing all weeks
def createTurnstileDataFrame(list_of_weeks):
    df = pd.DataFrame()
    df_chunk_list = []
    for date in list_of_weeks:
        df_chunk = readTurnstileData(date)
        df_chunk_list.append(df_chunk)
    df = pd.concat(df_chunk_list)
    return df

# Clean up date/time info
def formatDateTime(df):
    """
    This function converts the date and time into DateTime format in a single column
    and deletes the unformatted date and time columns
    
    Note: only run once per DataFrame, will result in error otherwise
    ---
    input: DataFrame
    output: DataFrame
    """
    
    #convert date and time to DateTime format in a single column
    df['concat_date_time'] = df['date'] + ' ' + df['time']
    df['date_time'] = pd.to_datetime(df.concat_date_time,format = '%m/%d/%Y %H:%M:%S')
    
    #delete unformatted date and time columns
    del df['concat_date_time']
    del df['date']
    del df['time']
    return df

def read_and_format_turnstile_data(list_of_weeks):
    """
    This function reads in Turnstile data from online and returns a DataFrame with 
    with the date and time information converted to a single DateTime column
    ---
    input: link
    output: DataFrame
    """
    df = createTurnstileDataFrame(list_of_weeks)
    df = formatDateTime(df)
    return df

In [84]:
# Read in the data from the specified weeks
orig_df = read_and_format_turnstile_data(weeks)

In [85]:
# Create a column for weekday
orig_df['weekday'] = orig_df[['date_time']].apply(lambda x: x['date_time'].dayofweek,axis=1)

In [86]:
# Create a column for the time of day
orig_df['hour'] = orig_df[['date_time']].apply(lambda x: x['date_time'].hour,axis=1)

In [87]:
# Create a column for year
orig_df['year'] = orig_df[['date_time']].apply(lambda x: x['date_time'].year,axis=1)

In [88]:
# Create a column with unique ID for each turnstile
orig_df['turnstile_id'] = orig_df.groupby(['control_area','unit','scp','station','year']).ngroup()

In [93]:
# Reset indexing on the DataFrame
orig_df.reset_index(inplace=True)

In [95]:
# Delete columns that won't be used
orig_df.drop(columns=['index','control_area','unit','scp','line_name','division','desc'],inplace=True)

In [104]:
# Save raw data to access easily later
with open('data/orig_df.pickle', 'wb') as to_write:
    pickle.dump(orig_df, to_write)

## Step 2ish: Read in Station Location Data

## Step 2.5: read in data from pickle

In [4]:
# Read in the data from the pickle file if you are starting here
# If you started from the beginning of the notebook, comment out the code below

with open('data/orig_df.pickle','rb') as read_file:
    orig_df = pickle.load(read_file)
df = orig_df

## Step 3: Organizing the Data

In [5]:
df.shape

(2409320, 8)

In [ ]:
# Create columns for latitude/longitude and borough of each turnstile

## Step 3: Cleaning the Data

In [106]:
# Sort DataFrame by turnstile and date
df.sort_values(['turnstile_id','date_time'],inplace=True)
df.reset_index(drop=True,inplace=True)

# Find entries and exits differences per turnstile
df['exit_counts'] = abs(df.groupby('turnstile_id').exits.diff())
df['entry_counts'] = abs(df.groupby('turnstile_id').entries.diff())

In [107]:
df.describe()

,entries,exits,weekday,hour,year,turnstile_id,exit_counts,entry_counts
count,2.409320e+06,2.409320e+06,2.409320e+06,2.409320e+06,2.409320e+06,2.409320e+06,2.399633e+06,2.399633e+06
mean,4.021208e+07,3.306011e+07,2.990220e+00,1.112374e+01,2.018507e+03,4.856233e+03,2.506554e+04,3.522321e+04
std,2.080115e+08,1.924366e+08,1.997990e+00,6.925758e+00,4.999505e-01,2.802334e+03,5.185810e+06,6.675077e+06
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.018000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.587520e+05,2.037400e+05,1.000000e+00,5.000000e+00,2.018000e+03,2.445000e+03,9.000000e+00,1.100000e+01
50%,2.426018e+06,1.377258e+06,3.000000e+00,1.100000e+01,2.019000e+03,4.840000e+03,5.300000e+01,7.700000e+01
75%,6.862688e+06,4.774284e+06,5.000000e+00,1.700000e+01,2.019000e+03,7.292000e+03,1.740000e+02,2.550000e+02
max,2.130144e+09,2.145850e+09,6.000000e+00,2.300000e+01,2.019000e+03,9.686000e+03,2.145033e+09,2.088495e+09


Max values on exit and entry counts are way too high

In [108]:
def findOutliers(df,column):
    """
    Returns outliers above the max limit for a column in a dataframe
    Adjust outlier cutoff to q75 + 4*iqr to include more data
    ---
    input: DataFrame, column
    output: DataFrame
    """
    q25,q50,q75 = df[column].quantile(q=[0.25,0.5,0.75])
    iqr = q75-q25
    #max limits to be considered an outlier
    max = q75 + 4*iqr
    #identify the points
    outlier_mask = [True if x > max else False for x in df[column]]
    print('{} outliers found out of {} data points, {}% of the data'.format(sum(outlier_mask),len(df[column]),100*(sum(outlier_mask)/len(df[column]))))
    return outlier_mask

In [109]:
#Get outliers for entries
print('Entry: ')
df['entry_outliers'] = findOutliers(df,'entry_counts')

#Get outliers for exits
print('\nExit:')
df['exit_outliers'] = findOutliers(df,'exit_counts')

#DataFrame with entry outliers removed
clean_df_entries = df.loc[~df['entry_outliers']]
print('\n{} points on entry side left after removing entry_counts outlier points'.format(clean_df_entries.shape[0]))

#DataFrame with exit outliers removed
clean_df_exits = df.loc[~df['exit_outliers']]
print('{} points on exit side left left after removing exit_counts outlier points'.format(clean_df_exits.shape[0]))


Entry: 
64486 outliers found out of 2409320 data points, 2.676522836318961% of the data

Exit:
92339 outliers found out of 2409320 data points, 3.832575166437003% of the data

2344834 points on entry side left after removing entry_counts outlier points
2316981 points on exit side left left after removing exit_counts outlier points


In [110]:
# Eliminate Null values
# Delete rows with null values for entry_counts
clean_df_entries = clean_df_entries[~clean_df_entries.entry_counts.isnull()]
print('{} points left after removing entry_counts Nan values'.format(clean_df_entries.shape[0]))

# Delete rows with null values for exit_counts
clean_df_entries = clean_df_entries[~clean_df_entries.exit_counts.isnull()]
print('{} points left after removing entry_counts Nan values'.format(clean_df_exits.shape[0]))


2335147 points left after removing entry_counts Nan values
2316981 points left after removing entry_counts Nan values


In [111]:
# Merge the two DataFrames
clean_df = clean_df_entries.merge(clean_df_exits,left_on=list(clean_df_entries.columns), \
                                  right_on=list(clean_df_exits.columns),how='inner')
#Add a column for total traffic at each turnstile
clean_df['total_traffic'] = clean_df['entry_counts'] + clean_df['exit_counts']

In [112]:
print('{} rows left after cleaning the data, {}% of the original'.format(clean_df.shape[0],100*(clean_df.shape[0]/df.shape[0])))

2284157 rows left after cleaning the data, 94.80504872744177% of the original


In [113]:
# Delete outlier ID columns
# Only run this cell once
del clean_df['entry_outliers']
del clean_df['exit_outliers']

In [114]:
# Save cleaned data to access easily later
with open('data/clean_df.pickle', 'wb') as to_write:
    pickle.dump(clean_df, to_write)


## Step 4.5: Read in the cleaned data

In [51]:
# Read in the data from the pickle file if you are starting here
# If you started from the beginning of the notebook, comment out the code below

import pickle

with open('data/clean_df.pickle','rb') as read_file:
    clean_df = pickle.load(read_file)